In [1]:
# Task: locate images in an xml file and download them

In [2]:
import lxml

In [3]:
from lxml import etree

In [4]:
tree2print = lxml.etree.parse("record_kuniweb_1109058.xml")
pretty = lxml.etree.tostring(tree2print, encoding="unicode", pretty_print=True)
print(pretty)

<mets:mets xmlns:mets="http://www.loc.gov/METS/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:mods="http://www.loc.gov/mods/v3" xmlns:dv="http://dfg-viewer.de/" xmlns:xlink="http://www.w3.org/1999/xlink" xsi:schemaLocation="http://www.loc.gov/mods/v3 http://www.loc.gov/standards/mods/v3/mods-3-7.xsd http://www.loc.gov/METS/ http://www.loc.gov/standards/mets/version17/mets.v1-7.xsd">
   <mets:metsHdr CREATEDATE="2020-09-10T08:51:57">
      <mets:agent ROLE="CREATOR" TYPE="ORGANIZATION">
         <mets:name>Verbundzentrale des GBV (VZG)</mets:name>
      </mets:agent>
   </mets:metsHdr>
   <mets:dmdSec ID="dmd_kuniweb_1109058_1">
      <mets:mdWrap MDTYPE="MODS">
         <mets:xmlData>
            <mods:mods>
               <mods:titleInfo>
                  <mods:title>"Wasserträger" aus Peru</mods:title>
               </mods:titleInfo>
               <mods:typeOfResource>three dimensional object</mods:typeOfResource>
               <mods:genre valueURI="http://d-nb.inf

In [5]:
import xml.etree.ElementTree as ET
tree = ET.parse('record_kuniweb_1109058.xml')
root = tree.getroot()

In [6]:
# Work on reading xml with namespaces

In [7]:
root.tag

'{http://www.loc.gov/METS/}mets'

In [8]:
root.attrib

{'{http://www.w3.org/2001/XMLSchema-instance}schemaLocation': 'http://www.loc.gov/mods/v3 http://www.loc.gov/standards/mods/v3/mods-3-7.xsd http://www.loc.gov/METS/ http://www.loc.gov/standards/mets/version17/mets.v1-7.xsd'}

In [9]:
agent = root.find(".//{http://www.loc.gov/METS/}agent")

In [10]:
agent.attrib

{'ROLE': 'CREATOR', 'TYPE': 'ORGANIZATION'}

In [11]:
ns = {'mets':'http://www.loc.gov/METS/'}

In [12]:
agent = root.find('.//mets:agent', ns)

In [13]:
agent.attrib

{'ROLE': 'CREATOR', 'TYPE': 'ORGANIZATION'}

In [14]:
# print mets:fileSec

In [15]:
file_sections = root.findall('./mets:fileSec', ns)

In [16]:
import requests
import io
from PIL import Image
import os
import hashlib

In [17]:
# this is the original function, as I found it online
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [18]:
# this is my version of the function
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file)
        file_path = os.path.join(folder_path,image_name)
        with open(file_path, 'wb') as f:
            image.save(f)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [19]:
for file_section in file_sections:
    files = file_section.findall('.//mets:file', ns)
    for file in files:
        file_location = file.find('./mets:FLocat', ns)
        # get original uri
        image_uri = file_location.get('{http://www.w3.org/1999/xlink}href')
        
        # get reduced size image uri
        image_uri = image_uri.replace('full/full','full/!800,800')
        print('image URI: ' + image_uri)
        
        # set name of image file
        image_name_start = image_uri.find('kuniweb_') + 8
        image_name_end = image_name_start + 6
        suffix_index = image_name_end + 9
        image_name = image_uri[image_name_start:image_name_end+1] + '_' + image_uri[suffix_index] + '.jpg'
        print('image name: ' + image_name)
        
        # download image
        # persist_image('../../Coding_da_Vinci/CdV_Niedersachsen_2020/Datengeber/Datensets/Landesmuseum Hannover/Ethnologie/Mediendateien',image_uri)
        persist_image('images',image_uri)
        print('\n')

image URI: https://kulturerbe.niedersachsen.de/rest/image/record_kuniweb_1109058/00000001.jpg/full/!800,800/0/default.jpg
image name: 1109058_1.jpg
SUCCESS - saved https://kulturerbe.niedersachsen.de/rest/image/record_kuniweb_1109058/00000001.jpg/full/!800,800/0/default.jpg - as images/1109058_1.jpg


image URI: https://kulturerbe.niedersachsen.de/rest/image/record_kuniweb_1109058/00000002.jpg/full/!800,800/0/default.jpg
image name: 1109058_2.jpg
SUCCESS - saved https://kulturerbe.niedersachsen.de/rest/image/record_kuniweb_1109058/00000002.jpg/full/!800,800/0/default.jpg - as images/1109058_2.jpg


